# drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cleaning bengal or english
!pip install git+https://github.com/csebuetnlp/normalizer
# kenlm for ctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -r /content/drive/MyDrive/audio/whisper/requirements.txt

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-qbpvh35t
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-qbpvh35t
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=627086def06bef4cf6aef9f78420613b7c5e157fb6c31aaf7671b36d79f0de40
  Stored in directory: /tmp/pip-ephem-wheel-cache-nncv7ynx/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186457 sha256=3bc25208ca99db5baeec4aa339fcb04bf0b1cba

# import

In [3]:
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as tat
import datasets
from datasets import load_dataset, load_metric, Audio, Dataset
import os
import gc
import glob
from pathlib import Path

import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa
import gc
import jiwer
import pyctcdecode
import kenlm
import torch
import evaluate
from torcheval.metrics import WordErrorRate
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, pipeline
from bnunicodenormalizer import Normalizer
import warnings
warnings.filterwarnings('ignore')
torchaudio.set_audio_backend("soundfile")

In [6]:
import sys
sys.path.append("/content/drive/MyDrive/audio/whisper/")

# process data

In [ ]:
### hyper-parameters
SR = 16000

# 입력크기가 고정적일 때 가장 효과적인 연산 알고리즘을 찾아서 모델 돌릴 때 적용해
torch.backends.cudnn.benchmark = True
output_dir = "/content/drive/MyDrive/audio/models/output_1"

In [ ]:
sentences = pd.read_csv("/content/drive/MyDrive/audio/dataset/normalized.csv")
indexes = set(pd.read_csv("/content/drive/MyDrive/audio/dataset/indexes.csv/indexes.csv")['id'])
print(len(sentences))

# mozilla-foundation/common_voice_11_0 dataset은 제거
sentences = sentences[~((sentences.index.isin(indexes))&(sentences['split']=='train'))].reset_index(drop=True)
print(len(sentences))

963636
706689


In [ ]:
data_0 = sentences.loc[sentences['split']=='valid'].reset_index(drop=True)
valid_0 = data_0.sample(frac=0.5, random_state=2000)
train_0 = data_0[~data_0.index.isin(valid_0.index)]

data_1 = sentences.loc[sentences['split']=='train'].reset_index(drop=True).sample(frac=0.1, random_state=2000)
valid_1 = data_1.sample(frac=0.5, random_state=2000)
train_1 = data_1[~data_1.index.isin(valid_1.index)]

train = pd.concat([train_0, train_1], axis=0).sample(frac=1, random_state=2000).reset_index(drop=True)
valid = pd.concat([valid_0, valid_1], axis=0).sample(frac=1, random_state=2000).reset_index(drop=True)

del data_0, data_1, valid_0, valid_1, train_0, train_1
all_ids = sentences['id'].to_list()
train_ids = train['id'].to_list()
valid_ids = valid['id'].to_list()

# in kaggle notebook, validating is very time-consuming, so here I use a very small validation set, rather than 5667.
valid = valid.sample(n=2000, random_state=42)

print(len(all_ids))
print(len(train_ids))
print(len(valid_ids))

706689
48649
48649


# mp3 다 못옮겨서 조정하는 부분 #

In [ ]:
func = lambda x : os.path.isfile(f'/content/drive/MyDrive/audio/dataset/train_mp3s/{x}.mp3')
train["check"] = train["id"].apply(func)
valid["check"] = valid["id"].apply(func)

In [ ]:
train = train[train["check"]]
valid = valid[valid["check"]]
print(train.shape, valid.shape)

(2407, 5) (107, 5)


In [ ]:
train = train.reset_index(drop=True).loc[:1000]
valid = valid.reset_index(drop=True).loc[:20]
print(train.shape, valid.shape)

(56, 5) (21, 5)


In [ ]:
del all_ids, train_ids, valid_ids, sentences, indexes
gc.collect()

180

# whisper

# load and train

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

# processor = AutoProcessor.from_pretrained("sawradip/bengali-whisper-medium-tugstugi")
# model = AutoModelForSpeechSeq2Seq.from_pretrained("sawradip/bengali-whisper-medium-tugstugi")
# processor = AutoProcessor.from_pretrained("sadidul012/whisper-small-bengali")
# model = AutoModelForSpeechSeq2Seq.from_pretrained("sadidul012/whisper-small-bengali")


processor = AutoProcessor.from_pretrained("byoussef/whisper-small-KR")
model = AutoModelForSpeechSeq2Seq.from_pretrained("byoussef/whisper-small-KR")
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
print(len(sorted_vocab_dict))

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

50364


In [ ]:
class WhisperDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.pathes = df['id'].values # 오디오 파일 경로
        self.sentences = df['normalized'].values # normalized된 문장
        self.resampler = tat.Resample(32000, SR) # 리샘플링

    def __getitem__(self, idx):
        apath = f'/content/drive/MyDrive/audio/dataset/train_mp3s/{self.pathes[idx]}.mp3'
        waveform, sample_rate = torchaudio.load(apath, format="mp3") # 로드
        waveform = self.resampler(waveform) #리샘플링


        # hugging face format으로 만들어주기
        batch = dict()
        y = processor(waveform.reshape(-1), sampling_rate=SR).input_features[0]
        batch["input_features"] = y
        batch["labels"] = processor(text=self.sentences[idx], sampling_rate=SR).input_ids

        return batch

    def __len__(self):
        return len(self.df)

train_dataset = WhisperDataset(train)
valid_dataset = WhisperDataset(valid)

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = 160000 #10초로 짜르기
    max_length_labels: Optional[int] = 100# 100단어로 짜르기
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods

#         print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt", # pytorch tensor로 받기
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=self.padding,
            max_length=self.max_length_labels,
            pad_to_multiple_of=self.pad_to_multiple_of_labels,
            return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch



data_collator = DataCollatorCTCWithPadding(processor)

In [ ]:
model.freeze_encoder()

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    group_by_length=False,
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    # eval_accumulation_steps=60,
    evaluation_strategy="steps",
    # save_strategy="steps",
    num_train_epochs=1,
    fp16=True,
    # save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    warmup_steps=600,
#     save_total_limit=1,
#     load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    prediction_loss_only=False,
#     auto_find_batch_size=True,
    report_to="none",
)

In [ ]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions[0]
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=processor.feature_extractor,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
)

In [ ]:
trainer.train()

# save

In [ ]:
trainer.save_model(output_dir)
processor.save_pretrained(output_dir)

# inference

In [ ]:
MODEL = "/content/drive/MyDrive/audio/models/output_1/"
CHUNK_LENGTH_S = 20.1
BATCH_SIZE = 4
DATASET_PATH = '/content/drive/MyDrive/audio/dataset/kr_test/'



# load file path
files = list(glob.glob(DATASET_PATH + '/' + '*.wav'))
files += list(glob.glob(DATASET_PATH + '/' + '*.mp3'))
files.sort()

# load model
pipe = pipeline(task="automatic-speech-recognition",
                model=MODEL,
                tokenizer=MODEL,
                chunk_length_s=CHUNK_LENGTH_S, device=0, batch_size=BATCH_SIZE)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="ko", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
texts = pipe(files, generate_kwargs={"max_length": 260, "num_beams": 4})
print(texts)

[{'text': ' 야 이 바보야'}]


In [16]:
import imp
imp.reload(infer)

<module 'infer' from '/content/drive/MyDrive/audio/whisper/infer.py'>

In [17]:
infer.inference('ko', 260, True, 4)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'text': '야 이 바보야'}]